In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 

https://github.com/sivasahukar95/CodeStore/blob/master/Scraping%20data%20from%20imdb.ipynb 

https://www.youtube.com/watch?v=I5L3OJ-xtsw

In [2]:
#Top 1000, sorted by user rating ascending 

url = "https://www.imdb.com/search/title/?groups=top_1000&count=100&sort=user_rating,asc"

url2 = "https://www.imdb.com/list/ls048276758/"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')


In [3]:
title = []
year = []
duration = []
rating = []
metascore = []

votes = []
gross = []

description = []
director = []
stars = []

In [4]:
movie_data = soup.findAll('div', attrs= {'class': 'ipc-metadata-list-summary-item'})

In [8]:
for store in movie_data:
    name = store.h3.a

    #title.append(name)
    # year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(','').replace(')','')
    

In [7]:
print(title)

[]
